# ACC with stop&go

In [2]:
import carla, time, pygame, math, random, cv2
import numpy as np

pygame 2.6.1 (SDL 2.28.4, Python 3.7.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

world = client.get_world()
spectator = world.get_spectator()

## Some helpful functions used in this notebook

In [4]:
def move_spectator_to(transform, distance=5.0, x=0, y=0, z=4, yaw=0, pitch=-30, roll=0):
    back_location = transform.location - transform.get_forward_vector() * distance
    
    back_location.x += x
    back_location.y += y
    back_location.z += z
    transform.rotation.yaw += yaw
    transform.rotation.pitch = pitch
    transform.rotation.roll = roll
    
    spectator_transform = carla.Transform(back_location, transform.rotation)
    spectator.set_transform(spectator_transform)

def spawn_vehicle(vehicle_index=0, spawn_index=0, pattern='vehicle.*'):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter(pattern)[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

def draw_on_screen(world, transform, content='O', color=carla.Color(0, 255, 0), life_time=20):
    world.debug.draw_string(transform.location, content, color=color, life_time=life_time)

def spawn_camera(attach_to=None, transform=carla.Transform(carla.Location(x=1.2, z=1.2), carla.Rotation(pitch=-10)), width=800, height=600):
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', str(width))
    camera_bp.set_attribute('image_size_y', str(height))
    #camera_bp.set_attribute('fov', '120')
    #camera_bp.set_attribute('sensor_tick', '0')
    camera = world.spawn_actor(camera_bp, transform, attach_to=attach_to)
    return camera

## Sensors

Sensors are another type of actors, usually spawned attached to a vehicle, designed to retrieve data from the world. The type of data depends on the sensor, but it can range from RGB images, LiDAR scans or even collision information.

Sensors are divided into two main categories:

- **Regular sensors**: these sensors retrieve data at a fixed rate, usually every tick (e.g RGB cameras).
- **Trigger sensors**: these sensors only retrieve data when a certain condition is met (e.g collision sensors).

Sensors details can be found in the [CARLA documentation](https://carla.readthedocs.io/en/latest/ref_sensors/).

![Sensors](img/sensors.jpg)

### Live camera feed

We can also visualize the camera feed in real-time. This is useful for debugging and understanding what the camera sees.

We can use the `opencv` library to create a window and display the camera feed.

In [6]:

vehicle = spawn_vehicle()
camera = spawn_camera(attach_to=vehicle, transform=carla.Transform(carla.Location(x=-6, z=5), carla.Rotation(pitch=-30)))

video_output = np.zeros((600, 800, 4), dtype=np.uint8)
def camera_callback(image):
    global video_output
    video_output = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

camera.listen(lambda image: camera_callback(image))

#vehicle.set_autopilot(True)
control = carla.VehicleControl(throttle=1000)
vehicle.apply_control(control)

cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)

running = True

try:
    while running:
        if cv2.waitKey(1) == ord('q'):
            running = False
            break
        print("Actor transform: ", vehicle.get_transform())
        print("Actor forward vector (direction):", vehicle.get_transform().get_forward_vector())
        print("Actor velocity: ", vehicle.get_velocity(),", ",vehicle.get_velocity().length(),"m/s, ",vehicle.get_velocity().length()*3.6,"km/h")
        cv2.imshow('RGB Camera', video_output)
        
finally:
    cv2.destroyAllWindows()
    camera.destroy()
    vehicle.destroy()

Actor transform:  Transform(Location(x=-64.644394, y=24.471010, z=-0.075111), Rotation(pitch=0.068090, yaw=0.159195, roll=-0.000000))
Actor forward vector (direction) Vector3D(x=0.999995, y=0.002778, z=0.001188)
Actor velocity:  Vector3D(x=-0.001330, y=-0.000004, z=-1.060344) ,  1.060344934463501 m/s,  3.8172417640686036 km/h
Actor transform:  Transform(Location(x=-64.644394, y=24.471010, z=-0.075111), Rotation(pitch=0.068090, yaw=0.159195, roll=-0.000000))
Actor forward vector (direction) Vector3D(x=0.999995, y=0.002778, z=0.001188)
Actor velocity:  Vector3D(x=-0.001330, y=-0.000004, z=-1.060344) ,  1.060344934463501 m/s,  3.8172417640686036 km/h
Actor transform:  Transform(Location(x=-64.644394, y=24.471010, z=-0.075111), Rotation(pitch=0.068090, yaw=0.159195, roll=-0.000000))
Actor forward vector (direction) Vector3D(x=0.999995, y=0.002778, z=0.001188)
Actor velocity:  Vector3D(x=-0.001330, y=-0.000004, z=-1.060344) ,  1.060344934463501 m/s,  3.8172417640686036 km/h
Actor transform:

In [12]:
camera_transforms = [
    (carla.Transform(carla.Location(x=1.5, z=2.4)), (600, 300)),  # Front camera
    (carla.Transform(carla.Location(x=-0.5, y=-0.9, z=2.4), carla.Rotation(yaw=-135)), (200, 400)),  # Left side camera
    (carla.Transform(carla.Location(x=-0.5, y=0.9, z=2.4), carla.Rotation(yaw=135)), (200, 400)),  # Right side camera
    (carla.Transform(carla.Location(x=-1.5, z=2.4), carla.Rotation(yaw=180)), (600, 300))  # Rear camera
]

vehicle = spawn_vehicle()
#control = carla.VehicleControl(throttle=0.5)
#vehicle.apply_control(control)
#world.tick()
vehicle.set_autopilot(True)


# Spawn cameras and attach to vehicle
cameras = []
video_outputs = [np.zeros((600, 800, 4), dtype=np.uint8) for _ in range(4)]

def create_camera_callback(index):
    def camera_callback(image):
        global video_outputs
        video_outputs[index] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    return camera_callback

for i, transform in enumerate(camera_transforms):
    camera = spawn_camera(attach_to=vehicle, transform=transform[0], width=transform[1][0], height=transform[1][1])
    camera.listen(create_camera_callback(i))
    cameras.append(camera)

cv2.namedWindow('Front Camera', cv2.WINDOW_AUTOSIZE)
cv2.namedWindow('Left Side Camera', cv2.WINDOW_AUTOSIZE)
cv2.namedWindow('Right Side Camera', cv2.WINDOW_AUTOSIZE)
cv2.namedWindow('Rear Camera', cv2.WINDOW_AUTOSIZE)

running = True

try:
    while running:
        if cv2.waitKey(1) == ord('q'):
            running = False
            break
        cv2.imshow('Front Camera', video_outputs[0])
        cv2.imshow('Left Side Camera', video_outputs[1])
        cv2.imshow('Right Side Camera', video_outputs[2])
        cv2.imshow('Rear Camera', video_outputs[3])
finally:
    cv2.destroyAllWindows()
    for camera in cameras:
        camera.destroy()
    vehicle.destroy()

In [4]:
vehicle = spawn_vehicle()

gps_blueprint = world.get_blueprint_library().find('sensor.other.gnss')
gps_sensor = world.spawn_actor(gps_blueprint, carla.Transform(), attach_to=vehicle)
print(carla.Transform())

ROAD_X_MIN = -110
LONGITUDE_MAX = 0.001

latest_gps = {"latitude": 0.0, "longitude": 0.0}

def gps_callback(gps_data):
    global latest_gps
    latest_gps["latitude"] = gps_data.latitude
    latest_gps["longitude"] = gps_data.longitude

gps_sensor.listen(gps_callback)

try:
    while True:
        current_x = latest_gps["longitude"]
        print(f"Longitude: {current_x}", end='\r')

        if current_x > LONGITUDE_MAX:
            new_location = carla.Location(x=ROAD_X_MIN, y=vehicle.get_transform().location.y)
            vehicle.set_transform(carla.Transform(new_location, vehicle.get_transform().rotation))

        control = carla.VehicleControl(throttle=0.5)
        vehicle.apply_control(control)

        world.tick()
except KeyboardInterrupt:
    pass
finally:
    gps_sensor.destroy()
    vehicle.destroy()

Transform(Location(x=0.000000, y=0.000000, z=0.000000), Rotation(pitch=0.000000, yaw=0.000000, roll=0.000000))
